In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
!ls -lha *.h5ad
!md5sum *.h5ad

-rw-rw-r-- 1 ing ing  11M Nov 30 09:57 demo_GEX.h5ad
-rw-rw-r-- 1 ing ing 895K Nov 30 09:57 demo_receptor_scores.h5ad
4f731abe3b90ca20d3339dd01002df2b  demo_GEX.h5ad
854f81f8532faf36bd3fea2d788e4534  demo_receptor_scores.h5ad


In [3]:
master_palette = {
    "ACTA2_Fibroblast": "#ffbb78", 
    "MHCII_Macrophage": "#7f7f7f", 
    "Activated_Bcell": "#c5b0d5", 
    "CDH12_Epithelial": "#1f77b4", 
    "KRT6A_Epithelial": "#ff7f0e", 
    "Dendritic_cell": "#c49c94", 
    "Endothelial": "#17becf", 
    "FAP_Fibroblast": "#8c564b", 
    "Inflam_Macrophage": "#aec7e8", 
    "UPK_Epithelial": "#2ca02c", 
    "KRT13_Epithelial": "#9467bd", 
    "CD8T": "#e377c2", 
    "Naive_Tcell": "#ff9896", 
    "PDGFRB_Fibroblast": "#f7b6d2", 
    "PDPN_Fibroblast": "#98df8a", 
    "Plasma_Bcell": "#c7c7c7", 
    "Cycling_Epithelial": "#d62728", 
    "Treg": "#dbdb8d", 
}
# https://stackoverflow.com/a/29643643
def hex2rgb(h):
    h = h.lstrip('#')
    return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

print(hex2rgb(master_palette['ACTA2_Fibroblast']))

(255, 187, 120)


In [4]:
adata = sc.read_h5ad("demo_GEX.h5ad")
sc.pp.normalize_total(adata, target_sum=10000)

radata = sc.read_h5ad("demo_receptor_scores.h5ad")

subtypes = np.unique(radata.obs.subtype)
celltypes = np.unique(radata.obs.celltype)
print(subtypes)
print(celltypes)

radata.shape

['ACTA2_Fibroblast' 'CD20_Bcell' 'CD8T' 'CDH12_Epithelial'
 'Cycling_Epithelial' 'Dendritic_cell' 'Endothelial' 'FAP_Fibroblast'
 'Inflam_Macrophage' 'KRT13_Epithelial' 'KRT6A_Epithelial'
 'MHCII_Macrophage' 'Naive_Tcell' 'PDGFRB_Fibroblast' 'PDPN_Fibroblast'
 'Plasma_Bcell' 'Treg' 'UPK_Epithelial']
['Endothelial' 'Epithelial' 'Fibroblast' 'Lymphocyte' 'Myeloid']


(7328, 23)

In [5]:
visium_channels = !cat ../visium_channels_2.txt
# Add PDCD1LG2_PDCD1
visium_channels.append('PDCD1LG2_PDCD1')
print(len(visium_channels))

1211


In [6]:
ligand_receptor = pd.read_csv("../cabello_aguilar_LR_PMID.csv", index_col=0,header=0)
# Add PDCD1LG2_PDCD1
ligand_receptor.loc[ligand_receptor.index[-1]+1, :] = ['PDCD1LG2', 'PDCD1', '']
# Cut down to receptors we have scored
ligand_receptor = ligand_receptor.loc[ligand_receptor.receptor.isin(radata.var_names)]
ligands = np.unique(ligand_receptor.ligand.tolist())
print(len(ligands))
ligand_receptor

192


ligand receptor        PMIDs
9     ADAM10      AXL      7822279
14    ADAM15    ITGA5  9914169,952
20    ADAM17    ITGA5     14970227
23    ADAM23    ITGA5     10749942
31     ADAM9    ITGA3     15361064
...      ...      ...          ...
3115   WNT3A     LRP6  14739301,13
3121    WNT3     LRP6     18528331
3143   WNT7A     LRP6          NaN
3157     ZP3     EGFR          NaN
3170  LGALS8    ITGA3     10852818

[287 rows x 3 columns]

In [7]:
percent_cutoff = 0.05

diffrx = {}
for ct in celltypes:
    if ct == 'Endothelial': continue
    rd = radata[radata.obs.celltype == ct]
    sc.tl.rank_genes_groups(rd, groupby='subtype', method='wilcoxon')
    ct_subtypes = np.unique(radata.obs.subtype.values[radata.obs.celltype == ct])
    print(list(ct_subtypes))
    rdx = pd.DataFrame(rd.X.toarray() > 0, index=rd.obs_names, columns=rd.var_names)
    rdx['subtype'] = rd.obs.subtype
    rdx = rdx.groupby('subtype').mean()

    for st in ct_subtypes:
        expressed_receptors = rdx.columns[rdx.loc[st] > percent_cutoff].tolist() # Percent expression cutoff
        df = sc.get.rank_genes_groups_df(rd, st)
        # remove pvalue cutoff for demo
        df = df.query("logfoldchanges > 0.5")
#         df = df.query("logfoldchanges > 0.5")
        df = df.loc[df.names.isin(expressed_receptors)]
        df['subtype'] = [st] * df.shape[0]
        diffrx[st] = df.copy()
        
rd = radata[radata.obs.celltype.isin(['Endothelial', 'Fibroblast'])]
sc.tl.rank_genes_groups(rd, groupby='subtype', method='wilcoxon')
df = sc.get.rank_genes_groups_df(rd,'Endothelial')
df = df.query("logfoldchanges > 0.25")
df['subtype'] = ['Endothelial'] * df.shape[0]
diffrx['Endothelial'] = df.copy()

Trying to set attribute `.uns` of view, copying.


['CDH12_Epithelial', 'Cycling_Epithelial', 'KRT13_Epithelial', 'KRT6A_Epithelial', 'UPK_Epithelial']


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.uns` of view, copying.


['ACTA2_Fibroblast', 'FAP_Fibroblast', 'PDGFRB_Fibroblast', 'PDPN_Fibroblast']
['CD20_Bcell', 'CD8T', 'Naive_Tcell', 'Plasma_Bcell', 'Treg']


Trying to set attribute `.uns` of view, copying.


['Dendritic_cell', 'Inflam_Macrophage', 'MHCII_Macrophage']


In [8]:
sender = 'CDH12_Epithelial'
background_set = ['Epithelial'] # Background cells to use for ligand enrichment

send_ad = adata[adata.obs.celltype.isin(background_set), adata.var_names.isin(ligands)]
sc.tl.rank_genes_groups(send_ad, groupby='subtype', method='wilcoxon', n_genes=send_ad.shape[1])
sender_df = sc.get.rank_genes_groups_df(send_ad, sender)
# remove pvalue requirement for demo
sender_df = sender_df.query("logfoldchanges > 0.5")
sender_ligands = sender_df.names.tolist()

N=25

# receivers = ['CDH12_CDH18_Epithelial', 'DSG3_Epithelial', 'Intermediate_Epithelial', 
#              'KRT_Epithelial', 'Proliferating_Epithelial']
receivers = ['ACTA2_Fibroblast', 'FAP_Fibroblast', 'PDGFRB_Fibroblast', 'PDPN_Fibroblast'] 
# receivers = ['Endothelial']
# receivers = ['CD8T', 'Naive_Tcell', 'Treg']
# receivers = ['Activated_Bcell', 'Plasma_Bcell']
# receivers += ['Activated_Bcell', 'Late_Activation_CD8T', 'Naive_Tcell', 'Plasma_Bcell', 'Treg']
# receivers = ['APC_Macrophage', 'Dendritic_cell', 'Inflam_Macrophage']

# Track the matched ligand-receptors 
agreement = {}
agreement_n = {}
for receiver in receivers:
    receptor_set = diffrx[receiver].names.values[:N]
    receptor_set = diffrx[receiver].names.values
    matched = ligand_receptor.loc[ligand_receptor.receptor.isin( receptor_set )]
    ligand_set = np.unique(matched['ligand'].values)

    lx = adata[:, adata.var_names.isin(ligand_set)]

    dx = pd.DataFrame(lx.X.toarray() > 0, index=adata.obs_names, columns=lx.var_names)
    dx['subtype'] = adata.obs.subtype
    print(dx.shape)

    pct = dx.groupby('subtype').mean() > percent_cutoff

    hits = pct.columns[pct.loc[sender]].tolist()
    hits = [h for h in hits if h in sender_ligands]

    channels = []
    for h in hits:
        rr = matched.loc[matched.ligand == h].receptor.values
        for r in rr:
            if r in receptor_set:
                channels.append(f'{h}_{r}')
                
    channels = sorted([c for c in channels if c in visium_channels])
    
    agreement[receiver] = channels
    agreement_n[receiver] = len(channels)
    print(sender, '-->', receiver, channels, len(channels))

# sender_agreement[sender] = agreement

Trying to set attribute `.uns` of view, copying.


(7328, 32)
CDH12_Epithelial --> ACTA2_Fibroblast ['ADAM23_ITGA5', 'ANGPT1_ITGA5', 'FBN1_ITGA5', 'TGFB1_TGFBR1', 'TGFB2_TGFBR1', 'TNC_CNTN1', 'TNC_ITGA5', 'VEGFD_ITGA5'] 8
(7328, 37)
CDH12_Epithelial --> FAP_Fibroblast ['ADAM23_ITGA5', 'ANGPT1_ITGA5', 'FBN1_ITGA5', 'FGF1_NRP1', 'FGF2_NRP1', 'FGF7_NRP1', 'PYY_FAP', 'SEMA3A_NRP1', 'SEMA3D_NRP1', 'SEMA3E_NRP1', 'TNC_ITGA5', 'VEGFD_ITGA5'] 12
(7328, 18)
CDH12_Epithelial --> PDGFRB_Fibroblast ['PDAP1_PDGFRB', 'PDGFC_FLT1', 'PDGFC_PDGFRB', 'PDGFD_PDGFRB', 'SEMA3E_PLXND1', 'SEMA4A_PLXND1', 'SEMA4A_PLXND1', 'VEGFC_FLT1', 'VEGFD_FLT1'] 9
(7328, 71)
CDH12_Epithelial --> PDPN_Fibroblast ['ANOS1_FGFR1', 'CALM3_ABCA1', 'COL11A1_ITGA2', 'COL1A2_ITGA2', 'COL4A1_ITGA2', 'COL4A3_ITGA2', 'COL6A1_ITGA2', 'COL6A2_ITGA2', 'COL6A3_ITGA2', 'COL8A1_ITGA2', 'DUSP18_ITGA2', 'FGF10_FGFR1', 'FGF12_FGFR1', 'FGF13_FGFR1', 'FGF14_FGFR1', 'FGF1_FGFR1', 'FGF2_FGFR1', 'FGF7_FGFR1', 'HSPG2_ITGA2', 'LAMA1_ITGA2', 'LAMA2_ITGA2', 'LAMC3_ITGA2', 'NUCB2_ERAP1', 'OLFM2_ROBO2',

In [9]:
all_ligands = set()
all_receptors = set()
for k, vv in agreement.items():
    for v in vv:
        ligand, receptor = v.split('_')
        all_ligands = all_ligands.union(set([ligand]))
        all_receptors = all_receptors.union(set([receptor]))
        
sd = adata[adata.obs.celltype.isin(background_set), adata.var_names.isin(list(all_ligands))]
# Percent of cells with nonzero ligand expression in sender
sdx = pd.DataFrame(sd.X.toarray() > 0, index=sd.obs_names, columns=sd.var_names)
sdx['subtype'] = sd.obs.subtype
sdx = sdx.groupby('subtype').mean().loc[[sender]]

# Mean ligand expression in sender 
sdxe = pd.DataFrame(sd.X.toarray(), index=sd.obs_names, columns=sd.var_names)
sdxe['subtype'] = sd.obs.subtype
sdxe = sdxe.groupby('subtype').mean().loc[[sender]]
print(sdx.shape)

rd = radata[:, radata.var_names.isin(list(all_receptors))]
# Percent of cells with non-zero receptor expression
rdx = pd.DataFrame(rd.X.toarray() > 0, index=rd.obs_names, columns=rd.var_names)
rdx['subtype'] = rd.obs.subtype
rdx = rdx.groupby('subtype').mean()
rdx = rdx.loc[receivers, :]

# Mean receptor score
rdxe = pd.DataFrame(rd.X.toarray(), index=rd.obs_names, columns=rd.var_names)
rdxe['subtype'] = rd.obs.subtype
rdxe = rdxe.groupby('subtype').mean()
rdxe = rdxe.loc[receivers, :]
print(rdx.shape)

(1, 42)
(4, 15)


In [10]:
import os

receiver_celltype = 'Fibroblast'
outbase = f'{sender}__to__{receiver_celltype}'

if not os.path.exists(outbase):
    os.makedirs(outbase)

In [11]:
import seaborn as sns
reds = [tuple(int(ch * 255) for ch in c) for c in sns.color_palette('Reds', 10)]
blues = [tuple(int(ch * 255) for ch in c) for c in sns.color_palette('Blues', 10)]
print(reds)
# sns.palplot(reds)
# sns.palplot(blues)

[(254, 229, 218), (253, 207, 188), (252, 180, 153), (252, 149, 117), (251, 120, 88), (246, 89, 63), (236, 56, 42), (208, 29, 31), (181, 18, 24), (147, 10, 18)]


In [12]:
def get_receptor_color(rdxe, receptor, receiver, cmap):
    # Change color scale
#     max_val = np.max(rdxe.loc[:, receptor].values)
    max_val = np.max(rdxe.values)
    bins = np.linspace(0, max_val, 10)
    b = np.digitize(rdxe.loc[receiver, receptor], bins, right=True)
    color = cmap[b]
    color = f','.join([f'{c}' for c in color])
    return color
    
# print(get_receptor_color(rdxe, 'ABCA1', 'ACTA2_Fibroblast', blues))
    
    
def write_receptor_karyotype(f, hlf, txtf, start, semi_circle, cmap=blues):
    # Also keep track of where to place ligands
    ligand_order = []
    
    # And coordinates for the links
    receptor_coords = {}
    
    total_receptors_with_repeats = 0
    for receiver, channels in agreement.items():
        receiver_receptors = set([x.split('_')[1] for x in channels])
        total_receptors_with_repeats += len(receiver_receptors)

    print(total_receptors_with_repeats)
    area_per_receptor = int(semi_circle / total_receptors_with_repeats)
    print(area_per_receptor)

    celltype_start = start
    receptor_start = start
    for receiver, channels in agreement.items():
        receptor_coords[receiver] = {}
        
        receiver_receptors = [x.split('_')[1] for x in channels]
        ligs = [x.split('_')[0] for x in channels]
        printed_receptors = []
#         for receptor, ligand in zip(receiver_receptors, ligs):
        for ch in channels:
            ligand, receptor = ch.split('_')
            if ligand not in ligand_order:
                ligand_order.append(ligand)
                
            if receptor in printed_receptors:
                continue
            else:
                printed_receptors.append(receptor)
                
            receptor_end = receptor_start + area_per_receptor
            
            # Track the receptor coordinates on this receiver
            receptor_coords[receiver][receptor] = (receptor_start, receptor_end)
            
            color = get_receptor_color(rdxe, receptor, receiver, cmap)
            
            hl = f'{receiver} {receptor_start} {receptor_end} fill_color={color}\n'
            hlf.write(hl)
            
            txt = f'{receiver} {receptor_start} {receptor_end} {receptor} color={color}\n'
            txtf.write(txt)
            
            receptor_start = receptor_end
        # The value of receptor start is now where our current cell type should stop
        color = ','.join([f'{v}' for v in hex2rgb(master_palette[receiver])])
        celltype_line = f'chr - {receiver} {receiver} {celltype_start} {receptor_start} {color}\n'
        f.write(celltype_line)
        # start for the next cell type
        celltype_start = receptor_start
        
    return ligand_order, receptor_coords

In [13]:
def get_ligand_color(sdxe, ligand, sender, cmap):
    # Change color scale
#     max_val = np.max(rdxe.loc[:, receptor].values)
    max_val = np.max(sdxe.values)
    bins = np.linspace(0, max_val, 10)
    b = np.digitize(sdxe.loc[sender, ligand], bins, right=True)
    color = cmap[b]
    color = f','.join([f'{c}' for c in color])
    return color

# Find and place ligands 
def write_ligands(hlf, txtf, start, total_ticks, ligand_order, cmap=reds):
    # Track ligand positions on the sender
    ligand_coords = {}
    
    n_ligands = len(ligand_order)
    print(n_ligands)
    borders = np.linspace(start, total_ticks, n_ligands+1, dtype=int)
    # Go in the reverse order that we wrote the receptors:
    for i, ligand in enumerate(ligand_order[::-1]):
        ligand_start = borders[i]
        ligand_end = borders[i+1]
        color = get_ligand_color(sdxe, ligand, sender, reds)
        hl = f'{sender} {ligand_start} {ligand_end} fill_color={color}\n'
        txt = f'{sender} {ligand_start} {ligand_end} {ligand} color={color}\n'
        
        ligand_coords[ligand] = (ligand_start, ligand_end)

        hlf.write(hl)
        txtf.write(txt)
    
    return ligand_coords

In [14]:
# 0 = fully opaque
MAX_TRANSPARENCY=127

def draw_links(linkf, sdxe, rdxe, agreement, ligand_coords, receptor_coords, receiver_order=None):
    if receiver_order is None:
        receiver_order = list(agreemen.keys())
#     for receiver, channels in agreement.items():

    max_interaction = 0
    for receiver in receiver_order:
        channels = agreement[receiver]
        for ch in channels:
            ligand, receptor = ch.split('_')
            rexp = rdxe.loc[receiver, receptor]
            sexp = sdxe.loc[sender, ligand]
            i = rexp * sexp
            if i > max_interaction:
                max_interaction = i
            
    print(max_interaction) 
    for receiver in receiver_order:
        channels = agreement[receiver]
        
        color = ','.join([f'{v}' for v in hex2rgb(master_palette[receiver])])
        
        for ch in channels:
            ligand, receptor = ch.split('_')
            l_c = ligand_coords[ligand]
            r_c = receptor_coords[receiver][receptor]
            
            rexp = rdxe.loc[receiver, receptor]
            sexp = sdxe.loc[sender, ligand]
            i = rexp * sexp
#             trx = MAX_TRANSPARENCY - int(MAX_TRANSPARENCY * (i / max_interaction))
            trx = f'{min(1-(i / max_interaction)**2, 0.9):3.3f}'
            print(sender, ligand, receiver, receptor, i, trx)
#             spct = sdx.loc[sender, ligand]
#             hl = (l_c[1] - l_c[0]) / 2
#             lm = l_c[0] + ((l_c[1] - l_c[0]) / 2)
#             l0 = int(lm - hl*spct)
#             l1 = int(lm + hl*spct)
            l0 = l_c[0]
            l1 = l_c[1]
            
#             rpct = rdx.loc[receiver, receptor]
#             hr = (r_c[1] - r_c[0]) / 2
#             rm = r_c[0] + ((r_c[1] - r_c[0]) / 2)
#             r0 = int(rm - hr*rpct)
#             r1 = int(rm + hr*rpct)
            r0 = r_c[0]
            r1 = r_c[1]
            
            
            link = f'{sender} {l0} {l1} {receiver} {r0} {r1} color={color},{trx}\n'
            
            linkf.write(link)

In [15]:
outbase

'CDH12_Epithelial__to__Fibroblast'

In [16]:
'''
karyotype.txt

chr - [name] [name] start stop R,G,B 
# start stop in int
# R,G,B in uint8

> 
'''
total_ticks = 10000
semi_circle = int(total_ticks/2)
print(semi_circle)

total_celltypes = len(sdx.index) + len(rdx.index)
print(total_celltypes)

f = open(f'{outbase}/karyotype.txt', 'w+')
hlf = open(f'{outbase}/highlights.txt', 'w+')
txtf = open(f'{outbase}/genes.txt', 'w+')
linkf = open(f'{outbase}/links.txt', 'w+')

# Circos starts at 12 o'clock
ligand_order, receptor_coords = write_receptor_karyotype(f, hlf, txtf, 0, semi_circle)

# write the sending semi-circle
color = ','.join([f'{v}' for v in hex2rgb(master_palette[sender])])
line = f'chr - {sender} {sender} {semi_circle} {total_ticks} {color}\n'
print(line)
f.write(line)

# now write the sending ligands
ligand_coords = write_ligands(hlf, txtf, semi_circle, total_ticks, ligand_order)

receiver_order = list(agreement.keys())[::-1]
draw_links(linkf, sdxe, rdxe, agreement, ligand_coords, receptor_coords, receiver_order=receiver_order)

f.close()
hlf.close()
txtf.close()
linkf.close()

5000
5
16
312
chr - CDH12_Epithelial CDH12_Epithelial 5000 10000 31,119,180

42
11.739133
CDH12_Epithelial ANOS1 PDPN_Fibroblast FGFR1 2.7961812 0.900
CDH12_Epithelial CALM3 PDPN_Fibroblast ABCA1 2.327637 0.900
CDH12_Epithelial COL11A1 PDPN_Fibroblast ITGA2 3.8590014 0.892
CDH12_Epithelial COL1A2 PDPN_Fibroblast ITGA2 4.972165 0.821
CDH12_Epithelial COL4A1 PDPN_Fibroblast ITGA2 4.461956 0.856
CDH12_Epithelial COL4A3 PDPN_Fibroblast ITGA2 3.0881462 0.900
CDH12_Epithelial COL6A1 PDPN_Fibroblast ITGA2 2.2929392 0.900
CDH12_Epithelial COL6A2 PDPN_Fibroblast ITGA2 3.3324084 0.900
CDH12_Epithelial COL6A3 PDPN_Fibroblast ITGA2 2.726873 0.900
CDH12_Epithelial COL8A1 PDPN_Fibroblast ITGA2 3.3927438 0.900
CDH12_Epithelial DUSP18 PDPN_Fibroblast ITGA2 1.2770377 0.900
CDH12_Epithelial FGF10 PDPN_Fibroblast FGFR1 1.0416963 0.900
CDH12_Epithelial FGF12 PDPN_Fibroblast FGFR1 10.834737 0.148
CDH12_Epithelial FGF13 PDPN_Fibroblast FGFR1 4.617864 0.845
CDH12_Epithelial FGF14 PDPN_Fibroblast FGFR1 6.2568